In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import logging
from pathlib import Path

import pandas as pd

from utils import download_file, unzip_file

LOGGER = logging.getLogger()

# Getting the data

First, let's obtain the acme dataset, and unzip it adjacent to this file.

In [5]:
url = f"https://github.com/JamesOwers/ACME/raw/master/ACMEv1.0.zip"
download_file(url)
unzip_file("ACMEv1.0.zip")

The dataset should now exist at a directory called `ACMEv1.0/acme`, adjacent to this notebook. The contents are as follows:

* `clean` - contains the extracted clean excerpts as csv files
* `altered` - contains the excerpts as csv files altered by the degradations described in `metadata.csv`

`metadata.csv` describes:

* (the id number for) the type of degradation used for the alteration
* the path for the altered and clean files
* which split (train, valid, test) the file should be used in
* in which corpus and on what line the file is located

`degradation_ids.csv` is a mapping of degradation name to the id number used in `metadata.csv`

The `{train,valid,test}_cmd_corpus.csv` are command-based (note_on, note_off, shift) versions of the acme data more convenient for our provided pytorch Dataset classes avaialble in the [Midi Degradation ToolKit package](https://github.com/JamesOwers/midi_degradation_toolkit).

Similarly, `{train,valid,test}_pr_corpus.csv` are piano-roll-based versions of the acme data more convenient for our provided pytorch Dataset classes avaialble in the [Midi Degradation ToolKit package](https://github.com/JamesOwers/midi_degradation_toolkit).

# The data

First let's look at the way the excerpts are stored as csvs. We call each of these a Note DataFrame, or a `note_df`, and they are an ordered list of the notes which have occurred in the excerpt.

In [10]:
root = Path("ACMEv1.0", "acme")

# The Metadata